# Algorithmes de clustering - Non supervisé

Dans l'apprentissage automatique, les types d'apprentissage peuvent être classés en trois grandes catégories : 

1. L'apprentissage supervisé 
2. L'apprentissage non supervisé 
3. L'apprentissage semi-supervisé  

Les algorithmes appartenant à la famille de l'apprentissage non supervisé n'ont aucune variable à prédire liée aux données. Au lieu d'avoir une sortie, les données n'ont qu'une entrée qui serait constituée de plusieurs variables qui décrivent les données. C'est là qu'intervient le regroupement.

La mise en cluster consiste à regrouper un ensemble d'objets de telle sorte que les objets d'un même cluster se ressemblent davantage les uns les autres qu'avec les objets d'autres clusters. La similarité est une mesure qui reflète la force de la relation entre deux objets de données. Le regroupement est principalement utilisé pour l'exploration de données. Elle est utilisée dans de nombreux domaines tels que le Machine Learning, la reconnaissance de formes, l'analyse d'images, la recherche d'informations, la bio-informatique, la compression de données et l'infographie.

## K-Means 

Il existe de nombreux modèles de **clustering**. Nous allons passer en revue les plus populaires. Malgré sa simplicité, le **K-means** est largement utilisé pour le clustering dans de nombreuses applications de Dara Science, particulièrement utile si vous avez besoin de découvrir rapidement des aperçus à partir de données **non étiquetées**. Dans ce notebook, nous voyons comment utiliser le k-Means pour la segmentation de la clientèle.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Question 1:
- Importer numpy, pandas, matplotlib.pyplot, seaborn
- Importer le fichier Mall_customers.csv et le stocker dans df
- Afficher le nombre de lignes et de colonnes
- Afficher les statistiques des variables
- Afficher le typage des variables
- Afficher les 10 première lignes de df

In [ ]:
from sklearn.preprocessing import StandardScaler

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import os
import warnings

warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/kaggle/input/ml-training-vlib/Mall_Customers.csv')
df.head()

In [ ]:
df.info(), df.describe(), df.shape

# Question 2:
- Renommer les variables "Annual Income (k$)" en 'Income' et 'Spending Score (1-100)' en 'Score'. Utiliser .rename() de pandas pour cela. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html
- Afficher un pairplit selon le genre (Homme - Femme) https://seaborn.pydata.org/generated/seaborn.pairplot.html
- Que peut on dire sur la variable gender? Semble t'elle pertinente pour notre segmentation?

In [ ]:
df.rename(index=str, columns={'Annual Income (k$)': 'Income',
                              'Spending Score (1-100)': 'Score'}, inplace=True)
df.head()

In [ ]:
# Let's see our data in a detailed way with pairplot
X = df.drop(['CustomerID', 'Gender'], axis=1)
sns.pairplot(df.drop('CustomerID', axis=1), hue='Gender', aspect=1.5)
plt.show()

Réponse: Le graphique ci-dessus montre que le sexe n'a pas de rapport direct avec la segmentation de la clientèle. C'est pourquoi nous pouvons laisser tomber et passer à d'autres caractéristiques et c'est pourquoi nous allons désormais utiliser le paramètre X.

# Question 3:
![](https://dendroid.sk/wp-content/uploads/2013/01/kmeansimg-scaled1000.jpg)

- Importer Kmeans de sklearn. https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
- Nous voulons réaliser plusieurs Kmeans afin d'utiliser la règle du coude afin de définir le nombre de cluster. Réaliser 10 kmeans.

In [ ]:
from sklearn.cluster import KMeans

clusters = []

for i in range(1, 11):
    km = KMeans(n_clusters=i).fit(X)
    clusters.append(km.inertia_)
    
fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(x=list(range(1, 11)), y=clusters, ax=ax)
ax.set_title('Searching for Elbow')
ax.set_xlabel('Clusters')
ax.set_ylabel('Inertia')

# Annotate arrow
ax.annotate('Possible Elbow Point', xy=(3, 140000), xytext=(3, 50000), xycoords='data',          
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3', color='blue', lw=2))

ax.annotate('Possible Elbow Point', xy=(5, 80000), xytext=(5, 150000), xycoords='data',          
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3', color='blue', lw=2))

plt.show()

La méthode du coude nous dit de sélectionner le cluster lorsqu'il y a un changement significatif de l'inertie. Comme on peut le voir sur le graphique, on peut dire que cela peut être 3 ou 5. Voyons les deux résultats dans le graphique et décidons.

Principe :Une stratégie simple pour identifier le nombre de classes consiste à faire varier K et surveiller l’évolution de l’inertie intra-classes W. L’idée est de visualiser le «coude» où l’adjonction d’une classe ne correspond à rien dans la structuration des données.

Pour la suite nous choisirons K = 3.

# Question 4:
- Instancier la méthode avec les paramètres n_clusters = 3
- Lancer un Kmeans avec K = 3
- Afficher le scatter plot croisant "Income" et "Score" avec pour label (hue) "Labels"

In [ ]:
# 3 cluster
km3 = KMeans(n_clusters=3).fit(X)

X['Labels'] = km3.labels_
plt.figure(figsize=(12, 8))
sns.scatterplot(X['Income'], X['Score'], hue=X['Labels'], 
                palette=sns.color_palette('hls', 3))
plt.title('KMeans with 3 Clusters')
plt.show()

# Question 5:
- Instancier la méthode avec les paramètres n_clusters = 5
- Lancer un Kmeans avec K = 5
- Afficher le scatter plot croisant "Income" et "Score" avec pour label (hue) "Labels"
- Utiliser random_state = 42 afin d'avoir tout le temps les mêmes résultats

In [ ]:
# Let's see with 5 Clusters
km5 = KMeans(n_clusters=5, random_state = 42).fit(X)

X['Labels'] = km5.labels_
plt.figure(figsize=(12, 8))
sns.scatterplot(X['Income'], X['Score'], hue=X['Labels'], 
                palette=sns.color_palette('hls', 5))
plt.title('KMeans with 5 Clusters')
plt.show()

À en juger par les graphiques, on pourrait dire que 5 grappes semblent meilleures que les 3. Comme il s'agit d'un problème non supervisé, nous ne pouvons pas vraiment savoir avec certitude lequel est le meilleur dans la vie réelle, mais en examinant les données, on peut dire sans risque de se tromper que 5 est un bon choix. 

# Question 6:
Nous avons identifier des groupes cependant il faut analyser les différences entre ces groupes afin de pouvoir donner un nom plus explicite que "Cluster N". Pour cela nous allons réaliser des graphiques. Il est possible de comparer nos populations avec des tests statistiques afin de juger de la significativité des différences entre clusters.

- Afficher le swarmplot entre "Labels" et "Income"
- Afficher le swarmplot entre "Labels" et "Score"
- Analyser les graphiques afin de trouver des noms à nos clusters

In [ ]:
fig = plt.figure(figsize=(20,8))
ax = fig.add_subplot(121)
sns.swarmplot(x='Labels', y='Income', data=X, ax=ax)
ax.set_title('Labels According to Annual Income')

ax = fig.add_subplot(122)
sns.swarmplot(x='Labels', y='Score', data=X, ax=ax)
ax.set_title('Labels According to Scoring History')

plt.show()

Réponse: 
Nous pouvons maintenant analyser en détail nos 5 groupes :
- L'étiquette 0 correspond aux revenus élevés et dépenses faibles
- L'étiquette 1 correspond aux revenus moyens et dépenses moyennes
- L'étiquette 2 correspond aux revenu faibles et dépenses élevés
- L'étiquette 3 correspond aux revenus élevés et dépenses élevés
- L'étiquette 4 correspond aux revenu faibles et dépenses faibles

## Hierarchical Clustering

## Agglomerative
Nous nous pencherons sur une technique de clustering, à savoir la mise en cluster hiérarchique agglomérative. L'agglomération est l'approche ascendante qui est plus populaire que le regroupement par division. Nous utiliserons également le lien complet comme critère de liaison. 

La classe de regroupement agglomératif nécessitera deux entrées :
- n_clusters : Le nombre de grappes à former ainsi que le nombre de centroïdes à générer. 
- lien : Quel critère de liaison à utiliser. Le critère de lien détermine la distance à utiliser entre les ensembles d'observation. L'algorithme fusionnera les paires de grappes qui minimisent ce critère.

# Question 7:
- Importer AgglomerativeClustering de sklearn. https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html
- Instancier la méthode avec les paramètres n_clusters = 5
- Afficher avec le scatterplot la relation "Income" et "Score" avec les labels associés ('Labels' - hue)

In [ ]:
from sklearn.cluster import AgglomerativeClustering 

agglom = AgglomerativeClustering(n_clusters=5).fit(X)

X['Labels'] = agglom.labels_
plt.figure(figsize=(12, 8))
sns.scatterplot(X['Income'], X['Score'], hue=X['Labels'], 
                palette=sns.color_palette('hls', 5))
plt.title('Agglomerative with 5 Clusters')
plt.show()

### Dendrogram pour Agglomerative Hierarchical Clustering

![](https://miro.medium.com/max/2556/1*r1YriAFjwJokgcdtaodQAQ.png)

N'oubliez pas qu'une matrice de distance contient la distance entre chaque point et chaque autre point d'un ensemble de données. Nous pouvons utiliser la fonction distance_matrix, qui nécessite deux entrées. N'oubliez pas que les valeurs de distance sont symétriques, avec une diagonale de 0. C'est une façon de s'assurer que votre matrice est correcte.

# Question 8:
- Importer hierarchy de scipy - https://docs.scipy.org/doc/scipy/reference/cluster.hierarchy.html
- Importer distance_matrix de scipy - https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance_matrix.html
- Utiliser distance_matrix pour calculer la matrice des distances en les individus de X
- Affihcer la matrice et vérifier que vous avez des 0 sur la diagonale

In [ ]:
from scipy.cluster import hierarchy 
from scipy.spatial import distance_matrix 

dist = distance_matrix(X, X)
print(dist)

# Question 9:
- Utiliser hierarchy.linkage sur la matrice des distances calculées précédemment
- Stocker dans Z

In [ ]:
Z = hierarchy.linkage(dist)

Un regroupement hiérarchique est généralement visualisé sous la forme d'un dendrogramme, comme le montre la cellule suivante. Chaque fusion est représentée par une ligne horizontale. La coordonnée y de la ligne horizontale est la similitude des deux groupes qui ont été fusionnés. En remontant de la couche inférieure vers le nœud supérieur, un dendrogramme nous permet de reconstruire l'histoire des fusions qui ont abouti à la mise en cluster représentée. 

# Question 10:
- Afficherle dendogram de Z

In [ ]:
plt.figure(figsize=(18, 50))
dendro = hierarchy.dendrogram(Z, leaf_rotation=0, leaf_font_size=12, orientation='right')

## Density Based Clustering (DBSCAN)

![](https://miro.medium.com/max/1128/1*ejlV2WryiH4zGFP_KohEeA.png)

La plupart des techniques de regroupement traditionnelles, telles que les k-means, le regroupement hiérarchique, peuvent être utilisées pour regrouper des données sans supervision. 

Cependant, lorsqu'elles sont appliquées à des tâches avec des clusters de forme arbitraire, ou des clusters à l'intérieur d'un cluster, les techniques traditionnelles peuvent ne pas permettre d'obtenir de bons résultats. En d'autres termes, les éléments d'un même groupe peuvent ne pas être suffisamment similaires ou les performances peuvent être médiocres. En outre, le clustering basé sur la densité localise les régions de haute densité qui sont séparées les unes des autres par des régions de faible densité. La densité, dans ce contexte, est définie comme le nombre de points dans un rayon donné.

Dans cette partie, l'accent sera mis sur la manipulation des données et des propriétés de DBSCAN et sur l'observation du clustering qui en résulte. De plus Kmeans force la création d'un nombre K de clusters. Ainsi tout les individus seront assignés à un regroupement, même les outliers... DBSCAN prend en compte ce type d'indivdus et va les isoler dans un autre cluster.

### Modeling
DBSCAN est l'acronyme de Density-Based Spatial Clustering of Applications with Noise. Cette technique est l'un des algorithmes de mise en cluster les plus courants qui fonctionne sur la base de la densité de l'objet. L'idée générale est que si un point particulier appartient à une cluster, il doit être proche de nombreux autres points de ce cluster.

Il fonctionne sur la base de deux paramètres : Epsilon et points minimums  
- Epsilon : Détermine un rayon précis qui, s'il comprend un nombre suffisant de points, est appelé zone dense  
- minimumSamples : Détermine le nombre minimum de points de données que nous voulons dans un quartier pour définir un cluster.

# Question 11:
- Importer DBSCAN
- Utiliser les paramètres eps = 11 et min_samples = 6 puis fitté l'algorithme
- Affihcer le scatterplot pour le croisement "Income", "Score" avec les labels des clusters
- Analyse les résultats du DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN 

db = DBSCAN(eps=11, min_samples=6).fit(X)

X['Labels'] = db.labels_
plt.figure(figsize=(12, 8))
sns.scatterplot(X['Income'], X['Score'], hue=X['Labels'], 
                palette=sns.color_palette('hls', np.unique(db.labels_).shape[0]))
plt.title('DBSCAN with epsilon 11, min samples 6')
plt.show()


# Question 12:

- Lancer les 3 algorithmes avec les paramètres défini précédemment
- Pour chaque algorithmes afficher le graphique croisant "Income", "Score" avec les labels associés

In [ ]:
fig = plt.figure(figsize=(20,15))

##### KMeans #####
ax = fig.add_subplot(221)

km5 = KMeans(n_clusters=5).fit(X)
X['Labels'] = km5.labels_
sns.scatterplot(X['Income'], X['Score'], hue=X['Labels'], style=X['Labels'],
                palette=sns.color_palette('hls', 5), s=60, ax=ax)
ax.set_title('KMeans with 5 Clusters')


##### Agglomerative Clustering #####
ax = fig.add_subplot(222)

agglom = AgglomerativeClustering(n_clusters=5).fit(X)
X['Labels'] = agglom.labels_
sns.scatterplot(X['Income'], X['Score'], hue=X['Labels'], style=X['Labels'],
                palette=sns.color_palette('hls', 5), s=60, ax=ax)
ax.set_title('Agglomerative with 5 Clusters')


##### DBSCAN #####
ax = fig.add_subplot(223)

db = DBSCAN(eps=11, min_samples=6).fit(X)
X['Labels'] = db.labels_
sns.scatterplot(X['Income'], X['Score'], hue=X['Labels'], style=X['Labels'], s=60,
                palette=sns.color_palette('hls', np.unique(db.labels_).shape[0]), ax=ax)
ax.set_title('DBSCAN with epsilon 11, min samples 6')

plt.tight_layout()
plt.show()

# Question 13:

Le coefficient de silhouette est une métrique usuelle pour évaluer la performance du clustering lorsqu'on ne connaît pas les vrais clusters.
Le coefficient de silhouette est calculé pour chaque échantillon : 

$$SC = \frac{b-a} {max(a, b)}$$

- a est la distance moyenne à tous les points du même cluster
- b est la distance moyenne à tous les autres points du cluster le plus proche.

La silhouette prend des valeurs de -1 (pire performance) à +1 (meilleure performance). Le score global est la moyenne de la silhouette.

le coefficient de silhouette est une mesure de qualité d'une partition d'un ensemble de données en clustering. Pour chaque point, son coefficient de silhouette est la différence entre la distance moyenne avec les points du même groupe que lui (cohésion) et la distance moyenne avec le points des autres groupes voisins (séparation). Si cette différence est négative, le point est en moyenne plus proche du groupe voisin que du sien : il est donc mal classé. A l'inverse, si cette différence est positive, le point est en moyenne plus proche de son groupe que du groupe voisin : il est donc bien classé.

Le coefficient de silhouette proprement dit est la moyenne du coefficient de silhouette pour tous les points.

- Importer metrics de sklearn
- Calculer la silhouette pour chaque algorithmes
- Lequel choisirez vous?

In [ ]:
from sklearn import metrics

print("Silhouette Coefficient Kmeans: %0.3f" % metrics.silhouette_score(X, km5.labels_, metric='sqeuclidean'))
print("Silhouette Coefficient Agglomerative: %0.3f" % metrics.silhouette_score(X, agglom.labels_, metric='sqeuclidean'))
print("Silhouette Coefficient DBSCAN: %0.3f" % metrics.silhouette_score(X, db.labels_, metric='sqeuclidean'))